In [1]:
import glob
import numpy as np
import os
import pandas as pd
import cv2
import shutil
from tqdm.notebook import tqdm

In [2]:
OBSERVE_LENGTH = 10
PREDICT_LENGTH = 5

TRAIN_SIZE = 20000
TEST_SIZE = int(TRAIN_SIZE / 5)

TRAIN_PREFIX = '/home/dataset/images/training/'
TEST_PREFIX = '/home/dataset/images/validation/'

def sample_data(src_csv, target_folder, train=True):
#     print(folder)
    data = pd.read_csv(src_csv).values
    
    data_length = len(data)
    
    start = np.random.randint(0, data_length-PREDICT_LENGTH-OBSERVE_LENGTH+1)
    clip_data = data[start:start+PREDICT_LENGTH+OBSERVE_LENGTH]
    
    X = clip_data[:OBSERVE_LENGTH]
    y = clip_data[-PREDICT_LENGTH:, -3:]
    
    np.save(target_folder+'X.npy', X)
    np.save(target_folder+'y.npy', y)
    
    #grab image content
    file_name = src_csv.split('/')[6]
    tar_name = ('_').join(file_name.split('_')[:2])
    seg_name = ('_').join(file_name.split('_')[2:])
    seg_name = seg_name[:-13]
    
    # VIEW from 0 - 4
    for view in range(5):
        if train:
            prefix = '/home/dataset/images/{}/training/'.format(view)
        else:
            prefix = '/home/dataset/images/{}/validation/'.format(view)
            
        path = prefix + tar_name + '/' + seg_name + '/'

        img_path = []
        for i in range(OBSERVE_LENGTH):
            img_path.append(path+'{}.npy'.format(i+start))
        
        with open(target_folder + 'img_path_{}.txt'.format(view), 'w') as f:
            f.write('\n'.join(img_path))
    
    
    

In [3]:
BOX_PTS = 9
training_csvs = glob.glob('/home/dataset/team2/training/training*/*_smooth_{}.csv'.format(BOX_PTS))

folder_idx = np.random.randint(0, len(training_csvs), TRAIN_SIZE)
# print(folder_idx)

train_path = '/home/dataset/training_observe_{}_predict_{}/'.format(OBSERVE_LENGTH, PREDICT_LENGTH)
if os.path.exists(train_path):
    shutil.rmtree(train_path)
os.mkdir(train_path)

for i in tqdm(range(TRAIN_SIZE)):
    idx = folder_idx[i]
    src_csv = training_csvs[idx]
    dst_folder = train_path + '{}/'.format(i)
    os.mkdir(dst_folder)
    sample_data(src_csv, dst_folder, train=True)

In [4]:
testing_csvs = glob.glob('/home/dataset/team2/validation/validation*/*_smooth_{}.csv'.format(BOX_PTS))

test_folder_idx = np.random.randint(0, len(testing_csvs), TEST_SIZE)

test_path = '/home/dataset/validation_observe_{}_predict_{}/'.format(OBSERVE_LENGTH, PREDICT_LENGTH)
if os.path.exists(test_path):
    shutil.rmtree(test_path)
os.mkdir(test_path)
    
    
for i in tqdm(range(TEST_SIZE)):
    idx = test_folder_idx[i]
    src_csv = testing_csvs[idx]
    dst_folder = test_path + '{}/'.format(i)
    os.mkdir(dst_folder)
    sample_data(src_csv, dst_folder, train=False)